## Imports


In [36]:
import pandas as pd
import numpy as np

# import saved model 
import pickle

# interactive 
from ipywidgets import interact, widgets  

## Import Model 

In [37]:
filename = "rfc_tune.pickle"
rfc_tune = loaded_model = pickle.load(open(filename, "rb"))

## Function to evaluate interactive input 

In [44]:
def yes_no_rfc(Applicant_Income, Coapplicant_Income,Loan_Amount
               , Loan_Amount_Term, Credit_History, Gender
               , Marital_Status, Dependents, Education_Graduate
               , Self_Employed, Property_Area): 
    
    total_income = np.log(Applicant_Income + Coapplicant_Income)
    emi = Loan_Amount / Loan_Amount_Term
    
    
    cat_feature = []
    
    if Credit_History == 'Yes': 
        cat_feature.append(1)
    else: 
        cat_feature.append(0)
    
    if Gender == 'Female':
        cat_feature.append(1)
        cat_feature.append(0)
    else:
        cat_feature.append(0)
        cat_feature.append(1)
    
    if Marital_Status == 'No':
        cat_feature.append(1)
        cat_feature.append(0)
    else: 
        cat_feature.append(0)
        cat_feature.append(1)
        
    if Dependents == 0:
        cat_feature.append(1)
        cat_feature.append(0)
        cat_feature.append(0)
        cat_feature.append(0)
    elif Dependents == 1: 
        cat_feature.append(0)
        cat_feature.append(1)
        cat_feature.append(0)
        cat_feature.append(0)
    elif Dependents == 2: 
        cat_feature.append(0)
        cat_feature.append(0)
        cat_feature.append(1)
        cat_feature.append(0)
    else: 
        cat_feature.append(0)
        cat_feature.append(0)
        cat_feature.append(0)
        cat_feature.append(1)  
    
    if Education_Graduate == 'Yes':
        cat_feature.append(1)
        cat_feature.append(0)
    else:
        cat_feature.append(0)
        cat_feature.append(1)
        
    if Self_Employed == 'No': 
        cat_feature.append(1)
        cat_feature.append(0)
    else:
        cat_feature.append(0)
        cat_feature.append(1)
        
    if Property_Area == 'Rural': 
        cat_feature.append(1)
        cat_feature.append(0)
        cat_feature.append(0)
    elif Property_Area == 'Semiurban': 
        cat_feature.append(0)
        cat_feature.append(1)
        cat_feature.append(0)
    else: 
        cat_feature.append(0)
        cat_feature.append(0)
        cat_feature.append(1)
        
    x_colunms = ['Credit_History', 'Total_Income_log', 'EMI', 'Gender_Female',
       'Gender_Male', 'Married_No', 'Married_Yes', 'Dependents_0',
       'Dependents_1', 'Dependents_2', 'Dependents_3+', 'Education_Graduate',
       'Education_Not Graduate', 'Self_Employed_No', 'Self_Employed_Yes',
       'Property_Area_Rural', 'Property_Area_Semiurban','Property_Area_Urban']
    
    x = [[cat_feature[0]],[total_income, emi], cat_feature[1:]]
    x_flat = [item for sublist in x for item in sublist]
    
    df_x = pd.DataFrame(x_flat).T
    df_x.columns = x_colunms
                        
    y = rfc_tune.predict(df_x)
    
    if y == 1:
        print('You will be approved')
        
    else: 
        print('You will not be approved')
        

In [41]:
# style to keep function parameters visable in the widget 
style = {'description_width': 'initial'}

app_in_range = widgets.IntSlider(description = 'Applicant Income:', min = 1, max = 8500
                                 , value = 3800, style = style) # , layout={"width": "500px"})

coapp_in_range = widgets.IntSlider(description = 'Coapplicant Income:', min = 0, max = 8500
                                   , value = 3800, style = style)
la_range = widgets.IntSlider(description = 'Loan Amount:', min = 1, max = 800
                             , value = 400, style = style)
lat_range = widgets.IntSlider(description = 'Loan Amount Term:',min = 6, max = 480
                              , value = 243 ,style = style)
ch_ = widgets.Dropdown(description = 'Credit History:', options = ['Yes', 'No']
                       , value = 'Yes', style = style)
gender_ = widgets.Dropdown(description = 'Gender:',options = ['Male', 'Female']
                           , value = 'Male', style = style)
marr_ = widgets.Dropdown(description = 'Marital Status:', options = ['Yes', 'No']
                         , value = 'Yes', style = style)
depends_ = widgets.Dropdown(description = 'Dependents:', options = ['0', '1', '2', '3+']
                            , value = '0',style = style)
education_ = widgets.Dropdown(descriptiom = 'Education Graduate:', options = ['Yes', 'No']
                              , value = 'Yes', style = style)
selfe_ = widgets.Dropdown(description = 'Self Employed:', options = ['Yes', 'No']
                          , value = 'Yes', style = style)
prop = widgets.Dropdown(description = 'Property Area:', options = ['Rural', 'Semiurban', 'Urban']
                        , value = 'Rural', style = style)

yes_or_no = widgets.interact(yes_no_rfc, Applicant_Income = app_in_range
                    , Coapplicant_Income = coapp_in_range ,Loan_Amount = la_range
                    , Loan_Amount_Term = lat_range, Credit_History = ch_, Gender = gender_
                    , Marital_Status = marr_, Dependents = depends_ 
                    , Education_Graduate = education_, Self_Employed = selfe_
                    , Property_Area = prop)


interactive(children=(IntSlider(value=3800, description='Applicant Income:', max=8500, min=1, style=SliderStyl…